In [28]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
files = os.listdir('./drive/MyDrive/Merged')
files

['gestures_test.csv', 'gestures_train.csv']

In [30]:
df_train = pd.read_csv('./drive/MyDrive/Merged/gestures_train.csv')
df_test = pd.read_csv('./drive/MyDrive/Merged/gestures_test.csv')
df_train

,label,a1,a2,ab,b1,b2,bc,c1,c2,cd,d1,d2,de,e1,e2,px,py,pz,x0,x1,x2,x3,x4,y0,y1,y2,y3,y4,z0,z1,z2,z3,z4
0,1,171.917694,132.738831,116.083084,168.437363,169.912949,2.744997,95.554222,125.426796,1.433788,98.878227,127.633797,0.321749,98.652611,128.144958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,172.026489,132.672134,116.079544,168.437363,169.912949,0.662870,95.559898,125.403061,2.515430,98.879311,127.635490,0.075647,98.653168,128.149155,-0.004623,0.011925,0.012167,59.335590,-0.010059,0.011242,0.008318,0.011974,41.752968,0.023743,0.069817,0.022484,0.015427,-60.686092,0.031288,-0.019450,0.028371,0.025351
2,1,172.049591,132.658401,116.078300,168.437317,169.913010,1.706766,95.556076,125.403023,1.034447,98.879570,127.635902,0.919123,98.653290,128.150757,-0.000923,0.002502,0.002552,0.001617,-0.002472,-0.004000,0.002121,0.002968,0.004364,0.005936,0.000786,0.005386,0.003067,0.007065,0.008162,0.010556,0.006170,0.006201
3,1,172.071579,132.642776,116.077629,168.437347,169.912949,0.455037,95.549484,125.403069,0.707033,98.879860,127.636314,1.301736,98.654289,128.151947,-0.000923,0.002495,0.002548,0.000597,-0.002464,-0.007114,0.002119,0.003578,0.004433,0.005936,-0.003082,0.005371,0.004326,0.007211,0.008144,0.013555,0.006168,0.004779
4,1,172.092804,132.627594,116.077316,168.437347,169.912933,2.034597,95.544991,125.404495,1.652241,98.880104,127.636703,0.060266,98.655296,128.153198,-0.003822,0.000137,0.001204,-0.002567,-0.008404,-0.008961,-0.001764,-0.000450,0.004295,0.002213,-0.005486,0.002556,0.000992,0.004589,0.001745,0.009327,0.003395,0.003071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161795,30,155.428940,174.586319,110.989807,168.322968,170.471313,0.891064,100.356743,141.077988,0.197881,101.467400,141.111938,2.246739,101.672791,141.356110,2.728947,0.310066,-0.266518,2.494905,3.028548,2.748571,2.845219,2.946190,0.300800,0.270683,0.573120,0.451969,0.419716,-0.081818,-0.191807,-0.038231,-0.073334,-0.167938
161796,30,155.546890,175.031281,111.164673,168.517853,170.631866,1.755110,100.357552,141.073151,0.849069,101.470001,141.112167,2.527119,101.694633,141.341766,3.301022,-0.286476,-0.032745,3.242838,2.957266,3.351566,3.343557,3.401852,-0.127975,0.304504,-0.441071,-0.423367,-0.339947,-0.481194,-0.374904,-0.335571,-0.306252,-0.319984
161797,30,155.883240,175.916321,111.659500,168.709732,170.790070,0.870190,100.386345,141.060516,1.251321,101.470955,141.110825,0.360201,101.697769,141.347458,1.644402,-0.627548,0.043282,1.661020,2.078659,1.358609,1.262947,1.341320,-0.388813,-0.135117,-1.058586,-0.907379,-0.691280,-0.401161,0.119175,-0.086823,-0.160332,-0.248985
161798,30,156.244843,176.664246,112.280891,168.823837,170.889343,1.974926,100.382736,141.060654,0.438103,101.471176,141.109833,1.286202,101.696381,141.344925,2.793831,-1.372391,-0.906891,2.365389,2.463518,2.307763,2.223122,2.245644,-0.584534,1.845512,-2.698456,-2.489361,-1.933666,-3.416515,-1.572324,-2.387756,-2.609138,-2.599831


In [21]:
# x = df_train.values #returns a numpy array
# min_max_scaler = MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df_train = pd.DataFrame(x_scaled)

# x = df_test.values #returns a numpy array
# min_max_scaler = MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df_test = pd.DataFrame(x_scaled)

In [31]:
def calculation(df_rows):
    res = {'omega0': [],
           'omega1': [],
           'omega2': [],
           'omega3': [],
           'omega4': [],
           'beta0': [],
           'beta1': [],
           'beta2': [],
           'beta3': [],
           'beta4': [],
           'u0':[],
           'u1':[],
           'u2':[],
           'u3':[],
           'u4':[],
           'u5':[],
           'v0':[],
           'v1':[],
           'v2':[],
           'v3':[],
           'v4':[],
           'v5':[],
           'z0':[],
           'z1':[],
           'z2':[],
           'z3':[],
           'z4':[],
           'z5':[],
           'gamma3': [],
           'gamma2': [],
           'gamma1': []
          }
    for index, row in df_rows.iterrows():
        #intra-finger angles
        res['gamma3'].append(row['ab'])
        res['gamma2'].append(row['bc'])
        res['gamma1'].append(row['cd'])
        
        #finger tip coordinates
        res['u0'].append(row['x0'])
        res['u1'].append(row['x1'])
        res['u2'].append(row['x2'])
        res['u3'].append(row['x3'])
        res['u4'].append(row['x4'])
        
        res['v0'].append(row['y0'])
        res['v1'].append(row['y1'])
        res['v2'].append(row['y2'])
        res['v3'].append(row['y3'])
        res['v4'].append(row['y4'])
        
        res['z0'].append(row['z0'])
        res['z1'].append(row['z1'])
        res['z2'].append(row['z2'])
        res['z3'].append(row['z3'])
        res['z4'].append(row['z4'])
        
        #palm coordinates
        res['u5'].append(row['px'])
        res['v5'].append(row['py'])
        res['z5'].append(row['pz'])
        
        #joint angles omega
        res['omega0'].append(row['e1'])
        res['omega1'].append(row['d1'])
        res['omega2'].append(row['c1'])
        res['omega3'].append(row['b1'])
        res['omega4'].append(row['a1'])
        
        #joint angles beta
        res['beta0'].append(row['e2'])
        res['beta1'].append(row['d2'])
        res['beta2'].append(row['c2'])
        res['beta3'].append(row['b2'])
        res['beta4'].append(row['a2'])
#     x = pd.DataFrame(res).values #returns a numpy array
#     min_max_scaler = MinMaxScaler()
#     x_scaled = min_max_scaler.fit_transform(x)
#     res = pd.DataFrame(x_scaled)
    res = pd.DataFrame(res)
#     display(res)
    return res.to_numpy()

In [32]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

def map_features(df):
    output_df = []
    output_label = []

    for i in range(200,len(df)+1,200):
        if i == 200:
            output_df.append(np.copy(calculation(df[:i])))
            output_label.append(df[:i]['label'][i-1]-1)
        else:
            output_df.append(np.copy(calculation(df[(i-200):i])))
            output_label.append(df[(i-200):i]['label'][i-1]-1)
    return np.array(output_df), np.array(output_label)


    

In [33]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))
x_train, y_train = map_features(df_train)

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

x_test, y_test = map_features(df_test)
num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [34]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# print(x_train)
y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (809, 200, 31)
y_train.shiape:  (809,)
x_test.shape:  (389, 200, 31)
y_test.shape:  (389,)


In [35]:
model = Sequential()
model.add(LSTM(units=200, input_shape=x_train.shape[1:], return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200, return_sequences=True ))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200, return_sequences=True ))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(30, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 200, 200)          185600    
_________________________________________________________________
batch_normalization_12 (Batc (None, 200, 200)          800       
_________________________________________________________________
dropout_12 (Dropout)         (None, 200, 200)          0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 200, 200)          320800    
_________________________________________________________________
batch_normalization_13 (Batc (None, 200, 200)          800       
_________________________________________________________________
dropout_13 (Dropout)         (None, 200, 200)          0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 200, 200)         

In [36]:
opt = tf.keras.optimizers.Adam(lr=0.0001)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

gestures = model.fit(x = x_train,
          y = y_train,
          epochs=100,
          validation_split=0.1, #split 10% of the trainning set for the validation set,
          batch_size=80,
#           validation_data = (x_test, y_test),
#           shuffle=True
         )

Epoch 1/100
10/10 [==============================] - 7s 220ms/step - loss: 3.6369 - accuracy: 0.1237 - val_loss: 3.3831 - val_accuracy: 0.0864
Epoch 2/100
10/10 [==============================] - 1s 96ms/step - loss: 1.9027 - accuracy: 0.4987 - val_loss: 3.3672 - val_accuracy: 0.0988
Epoch 3/100
10/10 [==============================] - 1s 96ms/step - loss: 1.3233 - accuracy: 0.6683 - val_loss: 3.3516 - val_accuracy: 0.1111
Epoch 4/100
10/10 [==============================] - 1s 96ms/step - loss: 1.1197 - accuracy: 0.6991 - val_loss: 3.3366 - val_accuracy: 0.1358
Epoch 5/100
10/10 [==============================] - 1s 98ms/step - loss: 0.9835 - accuracy: 0.7171 - val_loss: 3.3172 - val_accuracy: 0.1481
Epoch 6/100
10/10 [==============================] - 1s 95ms/step - loss: 0.8631 - accuracy: 0.7759 - val_loss: 3.2983 - val_accuracy: 0.1852
Epoch 7/100
10/10 [==============================] - 1s 96ms/step - loss: 0.7738 - accuracy: 0.7929 - val_loss: 3.2752 - val_accuracy: 0.1852
Epoch